In [38]:
import numpy as np
import matplotlib.pyplot as plt
import lmfit
%matplotlib inline

In [39]:
import os
import glob
import re
import sys

# print(os.path.abspath('.'))
# print(os.path.abspath('../core'))

py_SPC_path = os.path.normpath(r"C:\TRAVAIL\recherche\code\pySPC")
print(py_SPC_path)
sys.path.insert(0, py_SPC_path)

from core import Experiment

C:\TRAVAIL\recherche\code\pySPC


In [52]:
# Put directory of the spc files :
# os.chdir(r'C:\Users\MatthieuL\Documents\data\2018_05_24 deuxieme Calibration DLS\Or')
os.chdir(r'C:\Users\MatthieuL\Documents\data\2018_05_24 deuxieme Calibration DLS\Arg')
# os.chdir(r'C:\Users\MatthieuL\Documents\data\2018_05_24 deuxieme Calibration DLS\Fluo_3xdilue')

In [54]:
# Correlation parameters
max_correlTime_ms = 100
start_time_mu_s = 8
precision = 10

angle_list = []
CPS_list = []
tau_list = []
beta_list = []
coorelation_curves = []
mu2_list = []
time_axis = None

B_ini = 1
Beta_ini = 0.8
tau_ini = 350
mu_2_ini=0
mu_3_ini=0
mu_4_ini=0

In [55]:
def cumulant(t, B, beta, tau, mu2, mu3, mu4):
    return B + beta*np.exp(-t/tau)*((1 + mu2/2*t**2/tau**2 - mu3/6*t**3/tau**3 + mu4/24*t**4/tau**4)**2)

model = lmfit.Model(cumulant, nan_policy="propagate")
params = model.make_params(B=1, beta=0.7, tau=500, mu2=0, mu3=0, mu4=0)
# params['A0'].set(min=0.01, value=1)
params['mu2'].set(min=0, value=1e-3)
params['mu3'].set(min=0, value=1e-3)
params['mu4'].set(min=0, value=1e-3)



for spc_file in glob.glob("*.spc"):
    print(spc_file)
    angle_list.append(int(re.findall(r'\d+', spc_file)[0]))
    exp = Experiment.Experiment()
    exp.new_exp("file", [spc_file])

    CPS = exp.data.channels[0].CPS
    CPS_list.append(CPS)

    print("CPS : %f" % CPS)

    t1_tick = exp.data.channels[0].start_tick
    t2_tick = exp.data.channels[0].end_tick
    print("Calculating correlation")
    exp.DLS(0, 0, t1_tick, t2_tick, max_correlTime_ms, start_time_mu_s, precision)
    
    correlation_curve = exp.results.DLS_Measurements[0].data
    
    coorelation_curves.append(np.copy(exp.results.DLS_Measurements[0].data))
    time_axis = np.copy(exp.results.DLS_Measurements[0].timeAxis)
        
    print("Fitting")
    # exp.results.DLS_Measurements[0].set_model("Cumulant")
    # exp.results.DLS_Measurements[0].guess(idx_start=0, idx_end=-1)
    # exp.results.DLS_Measurements[0].set_params([B_ini, Beta_ini, tau_ini, mu_2_ini, mu_3_ini, mu_4_ini])
    # 
    # fitResults = exp.results.DLS_Measurements[0].fit(idx_start=0, idx_end=-1)
    # print(fitResults.best_values)
    # tau_list.append(fitResults.best_values['tau'])
    # beta_list.append(fitResults.best_values['beta'])
    # mu2_list.append(fitResults.best_values['mu2'])
    Gn = correlation_curve
    taus = time_axis
    
    fit_res = model.fit(Gn, t=taus, params=params)
    print('\nList of fitted parameters for %s: \n' % model.name)
    fit_res.params.pretty_print(colwidth=10, columns=['value', 'min', 'max'])
    tau_list.append(fit_res.best_values['tau'])
    beta_list.append(fit_res.best_values['beta'])
    mu2_list.append(fit_res.best_values['mu2'])    
    
    

    # print(fitResults.fit_report())
    # print(tau_list)

f = open("result.txt", "w")

i = 0
# print(angle_list)
for angle in angle_list:
    # print(angle)
    f.write("%d %d %f %f %f\n" % (angle, CPS_list[i], tau_list[i], beta_list[i], mu2_list[i]))
    i += 1
    

f.close()

f = open("correation_curve.txt", "w")

i = 0
j = 0

for j in range(np.size(time_axis)):
    # print("j = %d" % j)
    f.write("%f " % (time_axis[j]))
    i = 0
    for angle in angle_list:
        corelation_curve = coorelation_curves[i]
        # print(coorelation_curves[i])
        f.write("%f " % (corelation_curve[j]))
        i += 1
    f.write("\n")

f.close()




arg_100d_PolarRec.spc
CPS : 22618.926082
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B              1       -inf        inf
beta      0.7467       -inf        inf
mu2      0.02146          0        inf
mu3    0.0001985          0        inf
mu4      0.02144          0        inf
tau        125.4       -inf        inf
arg_105d_PolarRec.spc
CPS : 23466.242733
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7588       -inf        inf
mu2      0.04259          0        inf
mu3    4.549e-08          0        inf
mu4      0.01573          0        inf
tau        116.3       -inf        inf
arg_110d_PolarRec.spc
CPS : 23536.717013
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7388       -inf        inf
mu2      0.02009          0        inf
mu3    0.0007968          0        inf
mu4      0.01119          0        inf
tau        114.2       -inf        inf
arg_115d_PolarRec.spc
CPS : 24302.406217
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7509       -inf        inf
mu2      0.03162          0        inf
mu3     0.000448          0        inf
mu4      0.01273          0        inf
tau        110.6       -inf        inf
arg_120d_PolarRec.spc
CPS : 25435.554224
Calculating correlation


...

Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7445       -inf        inf
mu2      0.07267          0        inf
mu3     0.001698          0        inf
mu4    8.399e-06          0        inf
tau        178.6       -inf        inf
arg_80d_PolarRec.spc
CPS : 21863.969062
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B              1       -inf        inf
beta      0.7535       -inf        inf
mu2      0.05128          0        inf
mu3    0.0001907          0        inf
mu4    1.818e-06          0        inf
tau          163       -inf        inf
arg_85d_PolarRec.spc
CPS : 21528.150351
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7589       -inf        inf
mu2      0.08272          0        inf
mu3      0.00167          0        inf
mu4     0.002801          0        inf
tau        138.6       -inf        inf
arg_90d_PolarRec.spc
CPS : 21107.358370
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B          1.001       -inf        inf
beta      0.7229       -inf        inf
mu2      0.01356          0        inf
mu3    1.811e-06          0        inf
mu4      0.01981          0        inf
tau        149.9       -inf        inf
arg_95d_PolarRec.spc
CPS : 21901.517999
Calculating correlation


Fitting

List of fitted parameters for Model(cumulant): 

Name       Value        Min        Max
B              1       -inf        inf
beta      0.7253       -inf        inf
mu2     0.009873          0        inf
mu3    1.191e-05          0        inf
mu4      0.01475          0        inf
tau        133.6       -inf        inf
